In [ ]:
import pandas as pd
import numpy as np

"""
Clase para el Promedio Semestral (Ene - Jun y Jul - Dic)  
de análisis de las fuentes de datos de pasajeros y viajes.
"""

class PromedioSemestral():

    # Función constructora que recibe un diccionario con los datos de vuelos y pasajeros
    def __init__(self, UniónDatos):
        
        self.UniónDatos = UniónDatos

    # Método para la unión de conjuntos de datos (vuelos y pasajeros)
    def UnionDeDatos(self):
        
        # Concatenar los datos de vuelos y pasajeros de todos los años
        Unión_Vuelos = pd.concat(self.UniónDatos['Vuelos'])
        Unión_Pasajeros = pd.concat(self.UniónDatos['Pasajeros'])
        
        # Llamar al método que relaciona vuelos con pasajeros usando un ID en común
        return self.Relación(Unión_Vuelos, Unión_Pasajeros) 
    # Método para relacionar los datos de vuelos y pasajeros a través de un identificador común
    def Relación(self, Unión_Vuelos, Unión_Pasajeros):
        
        # Cambiar el nombre de la columna 'Cve_Cliente' a 'ID_Pasajero' para la unión
        Unión_Vuelos = pd.DataFrame(Unión_Vuelos).rename(columns={'Cve_Cliente':'ID_Pasajero'})
        
        # Hacer un merge (fusión) de los dos DataFrames utilizando 'ID_Pasajero' como llave
        RelaciónDeDatos = pd.merge(Unión_Pasajeros, Unión_Vuelos, on='ID_Pasajero')

        # Llamar al método para unir la información de las líneas aéreas
        return self.LíneasAéreas(RelaciónDeDatos)
    # Método para incorporar los datos de líneas aéreas en la relación de datos
    def LíneasAéreas(self, RelaciónDeDatos):
        
        # Cargar el archivo CSV que contiene información sobre las líneas aéreas
        LíneasAéreas = pd.read_csv('LineasAereas.csv')
        
        # Realizar un merge con la tabla de datos utilizando la clave de la línea aérea ('Cve_LA' y 'Code')
        Unión_LíneasAéreas = RelaciónDeDatos.merge(LíneasAéreas, left_on='Cve_LA', right_on='Code', how='left').fillna('Otra')

        # Crear un DataFrame con las columnas relevantes para el análisis
        Datos_Viaje = pd.DataFrame({
            'Fecha del viaje': pd.to_datetime(Unión_LíneasAéreas['Viaje']),
            'Clase': Unión_LíneasAéreas['Clase'],
            'Precio': Unión_LíneasAéreas['Precio'],
            'Ruta': Unión_LíneasAéreas['Ruta'],
            'Edad': Unión_LíneasAéreas['Edad'],
            'Línea Aérea': Unión_LíneasAéreas['Linea_Aerea'] 
        })

        # Llamar al método para calcular el promedio semestral
        return self.Promedio(Datos_Viaje)
    # Método para calcular el promedio de precios semestralmente y generar el archivo CSV
    def Promedio(self, Datos_Viaje):
        
        # Asegurarse que la columna 'Fecha del viaje' sea de tipo fecha
        Datos_Viaje['Fecha del viaje'] = pd.to_datetime(Datos_Viaje['Fecha del viaje'])
        
        # Crear columnas de Año y Semestre según la fecha del viaje
        Datos_Viaje['Año'] = Datos_Viaje['Fecha del viaje'].dt.year
        Datos_Viaje['Semestre'] = Datos_Viaje['Fecha del viaje'].dt.month.apply(lambda x: 1 if x <= 6 else 2)
        
        # Crear una columna que indique el nombre del semestre ('Ene-Jun' o 'Jul-Dic')
        Datos_Viaje['Periodo'] = Datos_Viaje['Semestre'].apply(lambda x: 'Ene-Jun' if x == 1 else 'Jul-Dic')
        
        # Agrupar por Año, Clase, Ruta, Línea Aérea, Semestre, y calcular el promedio del Precio
        promedio_por_semestre = Datos_Viaje.groupby(['Año', 'Clase', 'Ruta', 'Línea Aérea', 'Semestre', 'Periodo'])['Precio'].mean().reset_index()
        
        # Pivotear la tabla para que los semestres aparezcan como columnas
        Pivote = promedio_por_semestre.pivot_table(index=['Año', 'Clase', 'Ruta', 'Línea Aérea'],
                                                     columns='Periodo',
                                                     values='Precio').reset_index()
        
        # Renombrar las columnas para mayor claridad
        Pivote.columns = ['Año', 'Clase', 'Ruta', 'Línea Aérea', 'Precio Promedio (Ene-Jun)', 'Precio Promedio (Jul-Dic)']
        
        # Guardar el DataFrame resultante en un archivo CSV
        Pivote.to_csv('ArchivoSalida.csv', index=False)
        
        # Devolver el DataFrame final con los promedios por semestre
        return Pivote


# Cargar los datos de vuelos y pasajeros de 2016 y 2017
Vuelos2016 = pd.read_csv('vuelos2016.csv')
Vuelos2017 = pd.read_csv('vuelos2017.csv')
Pasajeros2016 = pd.read_csv('pasajeros2016.csv')
Pasajeros2017 = pd.read_csv('pasajeros2017.csv')

# Crear un diccionario que contiene los datos de vuelos y pasajeros para su análisis
UniónDatos = {
    'Vuelos': [Vuelos2016, Vuelos2017],
    'Pasajeros': [Pasajeros2016, Pasajeros2017]
}

# Instanciar la clase y realizar el análisis de datos
Análisis = PromedioSemestral(UniónDatos)

# Ejecutar el proceso de unión de datos y cálculo del promedio
Datos = Análisis.UnionDeDatos()

# Imprimir los resultados
print(Datos)


In [2]:
!jupyter nbconvert --to html PromedioSemestral.ipynb

[NbConvertApp] Converting notebook PromedioSemestral.ipynb to html
[NbConvertApp] Writing 284163 bytes to PromedioSemestral.html
